In [77]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [78]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [79]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [80]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,dqeHKSN-k0TBEhcPYUv8ry50QwP0BTwcFogU2HM5fO1v5Y...,ud4Y-zfjZlxjQ3cBpMoWwX1kqJ0gUkcu5UtwzWmk-UcCEo...,OP9BN_pQa8XVrr9lBrI4GX8bXkSKrZnXD3ph4JnEqIFp0F...,click my d,29,1656575110000,1,dqeHKSN-k0TBEhcPYUv8ry50QwP0BTwcFogU2HM5fO1v5Y...,click my d,642,I,112,79,False,False,True,False
1,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,MrWHYI6liJ4DJ07BK8x2oTa9cI6yNbE_9tj1PVwrUVoLoc8,JfpGAp6IYvz9ExTxKZ-R0T-oOATXhtI1KRRN5Y4nb6GkOm...,Liquid Kurumx,3440,1656567455000,114,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,Liquid Kurumx,1078,I,106,46,False,False,False,False
2,bicVkGlwpad_Ahy8HeUW7SCgtpu40D47FJi21MprClE3vm4,521JLtSaP9OJgbT79TDNIlZZycobCEEZNEtWOkzPBxs5Qw,2iaL68PHHTt846wEgYG-Toim_E0Hdw1obob-rUcEtiHCE-...,Ripple Overdrive,1635,1656312308000,64,bicVkGlwpad_Ahy8HeUW7SCgtpu40D47FJi21MprClE3vm4,Ripple Overdrive,743,I,136,101,False,False,False,False
3,hARKRxJanUz81P8rwTjSa8FBeH6Myi96d5hc8iTuGMYizaip,2fpMP8ao2DZ5oEB4yla-GGK1aYXf8-oIQd9LXQxlNQfSCD...,TsagUzmm9HLXpHKmYlkvWV-Zo0qTT7VfeCK5BKZhiiqMPr...,Marcel P,3545,1656585180000,19,hARKRxJanUz81P8rwTjSa8FBeH6Myi96d5hc8iTuGMYizaip,Marcel P,690,I,211,146,True,False,False,False
4,Gzedh6XF2T6y35wuGW5eKa0MQE8iyGpNJwj2mSMZAepcl2f-,-6z6VP5XJEvMWwQ8erU7LLSfibXto-UhZRzl_bX8KHTKZs...,gknC54sLodRm73oByx1NIFCSfm54xwWHjpKHygy5vOQsTq...,C9 k3soju,4293,1656551178000,39,Gzedh6XF2T6y35wuGW5eKa0MQE8iyGpNJwj2mSMZAepcl2f-,C9 k3soju,1334,I,256,168,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,d02KRlydgxxuMS7LH5z0Q3PHqpN0b1Bq7H5zvrxRigscjkQ,sPf2Ev7DnysqtfgttocOl0JspVyf5d45NXvJTUWSu0kGYjs,vZsyPcBDt7MNkkW3_8YzB-YGh6iipVJcnYbynJIwzEqy_A...,plumbum,984,1656220349000,65,d02KRlydgxxuMS7LH5z0Q3PHqpN0b1Bq7H5zvrxRigscjkQ,plumbum,617,I,143,97,False,False,False,False
110,mv-bQtHrJQVULXvSqPhsgXgQoUpLj7w8Ky_3g9S5zg5AZW...,E43KAncTyqyu6QhmJbR015xShMjAJWeI5Bq7MGrbBYaUfA...,dHUAGXGqGZYS46O_dTO_eezTZ15yy0XXQhC8KNtffdB23j...,vequeantethiha,29,1656586666000,1,mv-bQtHrJQVULXvSqPhsgXgQoUpLj7w8Ky_3g9S5zg5AZW...,vequeantethiha,565,I,88,62,False,False,True,False
111,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,RIzLPt0Yj1Nmx_V91Nq41KkK0jw0N8H6IJTCs3Smthlmc3...,XWwLJnEh-o7p_WMIa2mmoyRKwJoX9bDMucuEsb2PqrSZUK...,Nostereleven,4834,1656404629000,191,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,Nostereleven,753,I,148,108,False,False,True,False
112,rg2dNoXtS1jhecL99tPUbEbv-KX5c55Id8Nzx_jLfzUQ9lY,U7-um4aTa7a9WKtaP5SJo3YKVkvVbj0Y87C1tywb-iHKLw,7Sd2BCrssyyY8zDH-5bopHl6Zj9eqNKj5tsQqBHferw929...,kramsey,5067,1656600127356,214,rg2dNoXtS1jhecL99tPUbEbv-KX5c55Id8Nzx_jLfzUQ9lY,kramsey,449,I,97,66,False,False,True,False


In [81]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [82]:
len(uniq_matches_id)

3562

In [83]:
len(matches_asset)

5098

In [84]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [85]:
len(uniq_matches)

3562

In [86]:
matches_asset[300]['info']['game_version']

'Version 12.11.446.9344 (Jun 09 2022/13:23:39) [PUBLIC] <Releases/12.11>'

In [87]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [88]:
len(latest_matches)

1741

In [89]:
# latest_matches[0]['info']

In [90]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [91]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4358896508',
 'placement': 8,
 'augment0': 'TFT7_Augment_ShimmerscaleTrait',
 'augment1': 'TFT6_Augment_CyberneticImplants3',
 'augment2': 'TFT7_Augment_DragonHorde',
 'Set7_Bard': 1,
 'Set7_Cannoneer': 0,
 'Set7_Dragon': 2,
 'Set7_Evoker': 1,
 'Set7_Guardian': 1,
 'Set7_Guild': 1,
 'Set7_Mage': 0,
 'Set7_Mystic': 1,
 'Set7_Revel': 1,
 'Set7_Scalescorn': 0,
 'Set7_Shimmerscale': 2,
 'Set7_SpellThief': 1,
 'Set7_Trainer': 0,
 'TFT7_Braum': 1,
 'TFT7_Lulu': 2,
 'TFT7_Corki': 2,
 'TFT7_Corki_item0': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Corki_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Corki_item2': 'TFT7_Item_ShimmerscaleGoldmancersStaff',
 'TFT7_Sona': 1,
 'TFT7_Sona_item0': 'TFT_Item_Shroud',
 'TFT7_DragonGold': 2,
 'TFT7_DragonGold_item0': 'TFT_Item_RedBuff',
 'TFT7_DragonGold_item1': 'TFT7_Item_ShimmerscaleHeartOfGold',
 'TFT7_DragonGold_item2': 'TFT_Item_BrambleVest',
 'TFT7_Zoe': 1,
 'TFT7_Zoe_item0': 'TFT_Item_HextechGunblade',
 'TFT7_Zoe_item1': 'TFT_Item_SpearOfShoj

In [92]:
matches_league_df = pd.json_normalize(matches_array)

In [93]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,...,TFT7_Kayn_item2,TFT7_Sejuani_item2,TFT7_Senna_item0,TFT7_Taric_item2,TFT7_Leona_item2,TFT7_Heimerdinger_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2
0,NA1_4358896508,8,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CyberneticImplants3,TFT7_Augment_DragonHorde,1.0,0.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4358896508,3,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_Electrocharge3,TFT6_Augment_CyberneticUplink2,NaN,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4358896508,7,TFT7_Augment_EvokerEssenceTheft,TFT6_Augment_RadiantRelics,TFT6_Augment_ComponentGrabBag,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4358896508,2,TFT7_Augment_GadgetExpert,TFT6_Augment_VerdantVeil,TFT7_Augment_EvokerEssenceTheft,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4358896508,6,TFT6_Augment_Electrocharge2,TFT6_Augment_Electrocharge3,TFT6_Augment_Diversify2,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13923,NA1_4356399998,6,TFT6_Augment_ItemGrabBag1,TFT6_Augment_Ascension,TFT6_Augment_WindfallPlusPlus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13924,NA1_4356399998,8,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_ComponentGrabBag,TFT6_Augment_HighEndShopping,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13925,NA1_4356399998,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_RevelEmblem2,NaN,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13926,NA1_4356399998,2,TFT6_Augment_SecondWind1,TFT6_Augment_Diversify2,TFT6_Augment_CyberneticShell3,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,...,TFT7_Kayn_item2,TFT7_Sejuani_item2,TFT7_Senna_item0,TFT7_Taric_item2,TFT7_Leona_item2,TFT7_Heimerdinger_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2
0,NA1_4358896508,8,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CyberneticImplants3,TFT7_Augment_DragonHorde,1.0,0.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4358896508,3,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_Electrocharge3,TFT6_Augment_CyberneticUplink2,NaN,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4358896508,7,TFT7_Augment_EvokerEssenceTheft,TFT6_Augment_RadiantRelics,TFT6_Augment_ComponentGrabBag,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4358896508,2,TFT7_Augment_GadgetExpert,TFT6_Augment_VerdantVeil,TFT7_Augment_EvokerEssenceTheft,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4358896508,6,TFT6_Augment_Electrocharge2,TFT6_Augment_Electrocharge3,TFT6_Augment_Diversify2,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13923,NA1_4356399998,6,TFT6_Augment_ItemGrabBag1,TFT6_Augment_Ascension,TFT6_Augment_WindfallPlusPlus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13924,NA1_4356399998,8,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_ComponentGrabBag,TFT6_Augment_HighEndShopping,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13925,NA1_4356399998,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_RevelEmblem2,NaN,2.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13926,NA1_4356399998,2,TFT6_Augment_SecondWind1,TFT6_Augment_Diversify2,TFT6_Augment_CyberneticShell3,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)